In [1]:
import scvi
import scanpy as sc
import numpy as np
import pandas as pd

from data_utils import *
from tqdm.notebook import tqdm

import scipy.sparse as sp

import gc

Global seed set to 0
/home/gokul/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [12]:
qualities = np.logspace(-3, 0, 10)#[-1:]
replicates = 3
prefix = "../../latent-mutual-information/data/"

In [3]:
%%time

counts = sc.read_mtx(prefix + 'ltseq/stateFate_inVitro_normed_counts.mtx')

CPU times: user 2min 52s, sys: 6.85 s, total: 2min 59s
Wall time: 2min 55s


In [4]:
meta = pd.read_csv(prefix + 'ltseq/stateFate_inVitro_metadata.txt', sep='\t')
meta.head(3)

,Library,Cell barcode,Time point,Starting population,Cell type annotation,Well,SPRING-x,SPRING-y
0,d6_2_2,GCGTGCAA-AGAAGTTA,6.0,Lin-Kit+Sca1-,Undifferentiated,2,411.496,-96.190
1,d6_2_2,AAGGGACC-CTCGATGC,6.0,Lin-Kit+Sca1-,Undifferentiated,2,-587.462,-306.925
2,d6_2_2,CGTACCGA-AGCGCCTT,6.0,Lin-Kit+Sca1-,Monocyte,2,1429.805,-429.300


In [5]:
gene_names = pd.read_csv(prefix +'ltseq/stateFate_inVitro_gene_names.txt', 
                         header=None, sep='\t')
gene_names.head(3)

,0
0,0610006L08Rik
1,0610007P14Rik
2,0610009B22Rik


In [6]:
clones = sc.read_mtx(prefix+'ltseq/stateFate_inVitro_clone_matrix.mtx')

In [7]:
%%time
clone_ids = [np.argmax(clones.X[i, :]) for i in range(len(clones))]

CPU times: user 8.34 s, sys: 5.85 ms, total: 8.35 s
Wall time: 8.34 s


In [8]:
counts.obs['clone'] = clone_ids
counts.obs['time'] = meta['Time point'].values
counts.obs['well'] = meta['Well'].values
counts.obs['type'] = meta['Cell type annotation'].values
counts.obs['SPRING1'] = meta['SPRING-x'].values
counts.obs['SPRING2'] = meta['SPRING-y'].values

In [9]:
def nonzero_minimum_per_row(sparse_matrix):
    # Initialize an array to store the minimum values
    min_per_row = np.full(sparse_matrix.shape[0], np.inf)
    
    # Iterate through rows
    for i in range(sparse_matrix.shape[0]):
        # Get the non-zero elements in each row
        row_data = sparse_matrix.getrow(i).data
        if row_data.size > 0:
            min_per_row[i] = np.min(row_data)
    
    return min_per_row

nonzero_min_per_row = nonzero_minimum_per_row(counts.X)

reciprocal_min_per_row = 1 / nonzero_min_per_row

diagonal_matrix = sp.diags(reciprocal_min_per_row)

result_matrix = diagonal_matrix.dot(counts.X)

counts.X = result_matrix.astype(int)

In [13]:
for q in tqdm(qualities, desc='qualities'):

    for r in tqdm(range(replicates), desc='replicates', leave=False):

        np.random.seed(r)

        ad = get_ad_with_quality(counts, quality=q)
        ad.write_h5ad('data/LARRY_Q%.3f_rep%d.h5ad'%(q, r))

        print("Q %.3f counts per cell" %q)
        print(ad.raw.X.sum()/len(ad))

        del ad
        gc.collect()

qualities:   0%|          | 0/1 [00:00<?, ?it/s]

replicates:   0%|          | 0/3 [00:00<?, ?it/s]

Q 1.000 counts per cell
2586.629275634708
Q 1.000 counts per cell
2586.629275634708
Q 1.000 counts per cell
2586.629275634708
